## Mudança

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

dfs = []
path_dados = os.getenv("PATH_DADOS")
count = 0
for arquivo in os.listdir(path_dados):
    # Só foi nescessário pegar 5 meses aleatório pois eles já teriam 10000 casos
    if arquivo.endswith('.parquet') and count<5:
      dfs.append(pd.read_parquet(f'{path_dados}/{arquivo}'))
      count=+1


SEED = 130397

df = pd.concat(dfs,ignore_index=True)
df_rotulados = pd.read_csv(f'{path_dados}/dados_rotulados.csv')

# Caminho correto dos dados rotulados
arquivo_entrada = f'{path_dados}/dados_rotulados.csv'
arquivo_saida = f'{path_dados}/dados_rotulados_filtrados.csv'

# Verifica se o arquivo existe antes de continuar
if not os.path.exists(arquivo_entrada):
    print(f"Erro: O arquivo '{arquivo_entrada}' não foi encontrado. Verifique o caminho!")
else:
    # Carregando os dados rotulados
    df_rotulados = pd.read_csv(arquivo_entrada,index_col=0).reset_index(drop=True)

    # Exibe as primeiras linhas para entender o formato
    print("Primeiras linhas do DataFrame original:")
    #print(df_rotulados.head())

    print(df_rotulados.sentiment.unique())

    # Dicionário de mapeamento para converter sentimentos em valores numéricos
    mapeamento_sentimento = {
        'Positive': 1,
        'Negative': 0,
    }

    # Verifica se a coluna 'sentiment' existe no DataFrame
    if 'sentiment' not in df_rotulados.columns:
        print("Erro: A coluna 'sentiment' não existe no DataFrame.")
    else:
        # Aplicando o mapeamento na coluna 'sentiment'
        df_rotulados['sentiment'] = df_rotulados['sentiment'].map(mapeamento_sentimento)

        # Removendo valores neutros (0) e não classificados (NaN)
        df_rotulados = df_rotulados.dropna(subset=['sentiment'])

        # Convertendo a coluna 'sentiment' para inteiro (opcional)
        df_rotulados['sentiment'] = df_rotulados['sentiment'].astype(int)

        # Salvando o novo arquivo filtrado
        df_rotulados.to_csv(arquivo_saida, index=False)

        print(f"Arquivo filtrado salvo com sucesso: {arquivo_saida}")

df_rotulados = df_rotulados.rename(columns={'sentiment':'Cumprimento_Sentenca'})
df_rotulados_positivo  = df_rotulados[df_rotulados['Cumprimento_Sentenca']==1]

Primeiras linhas do DataFrame original:
['Negative' 'Positive' 'Neutral' nan]
Arquivo filtrado salvo com sucesso: dados/dados_rotulados_filtrados.csv


In [ ]:
from sklearn.model_selection import train_test_split

def split_holdout(df):
    """
    Separa um dataframe em conjuntos de treino (80%), validação (10%) e teste (10%).

    Parâmetros:
    df (DataFrame): O dataframe contendo os dados não rotulados.

    Retorna:
    tuple: (treino, validacao, teste)
    """

    # Separar 80% para treino e 20% para validação + teste
    treino, temp = train_test_split(df, test_size=0.2, random_state=42)

    # Separar 10% para validação e 10% para teste
    validacao, teste = train_test_split(temp, test_size=0.5, random_state=42)

    return treino, validacao, teste

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

def extrair_caracteristicas_de_dataframe(
    df: pd.DataFrame, tokenizer: AutoTokenizer, modelo: AutoModel
) -> pd.DataFrame:
    """Extrai características de um DataFrame contendo textos e rótulos.

    Args:
        df (pd.DataFrame): DataFrame contendo as colunas 'texto' e 'Cumprimento_Sentença'.
        tokenizer (AutoTokenizer): O tokenizer do Hugging Face.
        modelo (AutoModel): O modelo pré-treinado do Hugging Face.

    Returns:
        pd.DataFrame: DataFrame com colunas ['rotulo', 'texto', 'caracteristicas']
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo.to(device)
    caracteristicas = []

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        texto = row["texto"]
        rotulo = row.get("Cumprimento_Sentenca", 0)  # Assume 0 se não houver valor

        if pd.isna(texto):
            continue  # Ignorar linhas sem texto

        # Tokenizar o texto
        entradas = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
        entradas = {k: v.to(device) for k, v in entradas.items()}  # Mover inputs para a GPU

        # Passar o texto tokenizado pelo modelo para obter as saídas
        with torch.no_grad():
            saidas = modelo(**entradas)

        # Extrair os embeddings da última camada do modelo
        ultimos_estados_ocultos = saidas.last_hidden_state

        # Calcular a média dos estados ocultos para obter um vetor de características
        vetor_de_caracteristicas = (
            ultimos_estados_ocultos.mean(axis=1).squeeze().detach().cpu().numpy()
        )

        # Adicionar à lista de características
        caracteristicas.append((rotulo, texto, vetor_de_caracteristicas))

    return pd.DataFrame(caracteristicas, columns=["rotulo", "texto", "caracteristicas"])


# Amostragem e divisão dos DataFrames
tamanho = 10000
df_nao_rotulado = df.sample(tamanho - len(df_rotulados_positivo))
cumprimento_treino, cumprimento_validacao, cumprimento_teste = split_holdout(df_rotulados_positivo)
nao_rotulado_treino, nao_rotulado_validacao, nao_rotulado_teste = split_holdout(df_nao_rotulado)

# Carregar o modelo e o tokenizer do Hugging Face para o processamento de texto
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
modelo = AutoModel.from_pretrained("answerdotai/ModernBERT-base")

if 'caracteristicas_treino' in os.listdir():
  caracteristicas_treino = pd.read_parquet('caracterisiticas_treino.parquet')
else:
  caracteristicas_treino = pd.concat([
      extrair_caracteristicas_de_dataframe(cumprimento_treino, tokenizer, modelo),
      extrair_caracteristicas_de_dataframe(nao_rotulado_treino, tokenizer, modelo)
  ])
if 'caracteristicas_validacao.parquet' in os.listdir():
  caracteristicas_validacao = pd.read_parquet('caracterisiticas_validacao.parquet')
else:
  caracteristicas_validacao = pd.concat([
    extrair_caracteristicas_de_dataframe(cumprimento_validacao, tokenizer, modelo),
    extrair_caracteristicas_de_dataframe(df_rotulados[df_rotulados['Cumprimento_Sentenca']==0],tokenizer, modelo),
])
if 'caracteristicas_teste.parquet' in os.listdir():
  caracteristicas_teste = pd.read_parquet('caracterisiticas_teste.parquet')
else:
  caracteristicas_teste = pd.concat([
      extrair_caracteristicas_de_dataframe(cumprimento_teste, tokenizer, modelo),
      extrair_caracteristicas_de_dataframe(nao_rotulado_teste, tokenizer, modelo)
  ])


In [ ]:
caracterisiticas_treino = caracteristicas_treino.reset_index(drop=True)
caracterisiticas_validacao = caracteristicas_validacao.reset_index(drop=True)
caracterisiticas_teste = caracteristicas_teste.reset_index(drop=True)

caracteristicas_treino.to_parquet('caracterisiticas_treino.parquet')
caracteristicas_validacao.to_parquet('caracterisiticas_validacao.parquet')
caracteristicas_teste.to_parquet('caracterisiticas_teste.parquet')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Carregar os dados
caracteristicas_treino = pd.read_parquet('caracterisiticas_treino.parquet')

# Converter a coluna 'caracteristicas' para um array NumPy
caracteristicas = np.array(caracteristicas_treino['caracteristicas'].tolist())

# Aplicar PCA para reduzir a dimensionalidade para 3 componentes
pca = PCA(n_components=3)
caracteristicas_pca = pca.fit_transform(caracteristicas)

# Imprimir a variância explicada
print("Variância explicada por cada componente:", pca.explained_variance_ratio_)
print("Variância total explicada:", np.sum(pca.explained_variance_ratio_))

# Criar o gráfico 3D de dispersão
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(caracteristicas_pca[caracteristicas_treino['rotulo'] == 1, 0],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 1, 1],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 1, 2],
           label='Rótulo 1', marker='x')
ax.scatter(caracteristicas_pca[caracteristicas_treino['rotulo'] == 0, 0],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 0, 1],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 0, 2],
           label='Rótulo 0', marker='o',alpha=0.02)

ax.set_xlabel('Componente Principal 1')
ax.set_ylabel('Componente Principal 2')
ax.set_zlabel('Componente Principal 3')
ax.set_title('PCA das Características em 3D')
ax.legend()
plt.show()


In [ ]:
X_train = caracteristicas_treino['caracteristicas']
y_train = caracteristicas_treino['rotulo'].tolist()
X_valid = caracteristicas_validacao['caracteristicas']
y_valid = caracteristicas_validacao['rotulo'].tolist()


# Convert the combined labels to a new format
# If the label is 0 (unlabeled), convert it to -1
# If the label is 1 (positive), keep it as 1
y_train_formatted = np.array([-1 if x == 0 else 1 for x in y_train])

# Count the occurrences of each label (-1 and 1) in the formatted labels
# np.bincount counts the number of occurrences of each value in the array
# Since np.bincount expects non-negative integers, it will not work directly with -1
# To handle this, we can use a workaround by adding 1 to each element before counting
# This shifts the range to non-negative integers
counts = np.bincount(y_train_formatted + 1)

# Print the counts of -1 and 1
# counts[0] corresponds to the count of -1 (originally 0 in the shifted range)
# counts[2] corresponds to the count of 1 (originally 2 in the shifted range)
print(f"Count of -1 (unlabeled): {counts[0]}")
print(f"Count of 1 (positive): {counts[2]}")

In [ ]:
X_train = np.array(X_train.tolist())
X_valid = np.array(X_valid.tolist())

y_train = np.array(y_train)
y_valid = np.array(y_valid)

In [ ]:
from pulearn import ElkanotoPuClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, matthews_corrcoef
import helpers.classification

# Initialize the RandomForestClassifier
# n_jobs=-1: Use all available CPU cores for parallel processing
# random_state=271828: Seed for random number generator to ensure reproducibility
model = MLPClassifier(random_state=271828)

# Initialize the ElkanotoPuClassifier with the RandomForestClassifier as the base estimator
# hold_out_ratio=0.30: Ratio of positive samples to hold out for estimating P(s=1|y=1)
pu_estimator = ElkanotoPuClassifier(estimator=model, hold_out_ratio=0.10)

# Fit the PU classifier on the combined dataset
# X_train: Feature matrix containing both positive and unlabeled samples
# y_train_formatted: Labels formatted to -1 for unlabeled and 1 for positive samples
pu_estimator.fit(X_train, y_train_formatted)

# Predict labels for the validation set using the trained PU classifier
y_valid_pred = pu_estimator.predict(X_valid)

# Show the classification metrics
helpers.classification.print_classification_metrics(y_valid, y_valid_pred)